# Title of notebook

Brief 1-2 sentence description of notebook.

# Summary

- Experiment
    - Observing fighting behavior between mice in their home cage in different settings. Mice are categorized as winner and loser based on if they initiated the behavior or if they are recieving it. Recording sessions are usually from 30 minutes to a hour with cages of 2 to 6 mice.
- Data
    - Excel spreadsheet of recorded fighting behavior. The relevant columns are those of the "Date" of the recording, the "winner" of the interaction, and the "loser" of the interaction. Each row will be for one interaction between two mice.
    - There is a cage for each sheet of the spreadsheet
    - For each recording session, we will assume that the date will be specified for that first row. We will use this to seperate all the rows into seperate sessions.
- Purpose of this Jupyter Notebook
    - To calculate the Elo Score of each mice after each interaction. The mice start off with an Elo score of 1000. Elo scores are calculated with the formula from here: https://www.omnicalculator.com/sports/elo . Then we will plot the change of elo score across all interactions. With the number of interactions on the X-Axis and the current Elo score on the Y. There will be a line for mice.

In [1]:
import copy
import re
import os
import sys
import string
import glob
import ast
from collections import Counter
from collections import defaultdict
import warnings

In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import git
# Getting the path of the root directory so that we can import repo specific functions
git_repo_object = git.Repo('.', search_parent_directories=True)
git_repo_directory = git_repo_object.working_tree_dir

In [4]:
# Setting path so that we can import functions
sys.path.append(os.path.join(git_repo_directory, "src"))

In [5]:
os.path.join(git_repo_directory, "src")


'/nancy/projects/social_dominance_active_inference/src'

In [6]:
from elorating import calculation
from elorating import dataframe

In [7]:
# Increase size of plot in jupyter

plt.rcParams["figure.figsize"] = (18,10)

## Inputs & Data

Explanation of each input and where it comes from.

In [8]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"/root/work/" # where data is saved should always be shown in the inputs

## Inputs & Data

Explanation of each input and where it comes from.

In [9]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [10]:
WEIGHTS_PATH = "../../data/pilot_3/Reward_Training_Weights_C57vsCD1_Pilot3.csv"
ELO_RATING_HISTORY_GLOB = glob.glob("./proc/elo_rating_spread_sheets/*/*elo-rating-history.csv") 

PAIRED_RC = pd.read_csv("./proc/elo_rating_spread_sheets/reward_competition/reward_competition_grouped_by_pairs_cage_1_2_3_4_5_6_date_20221003_20221004.csv", index_col=0)
PAIRED_TT = pd.read_csv("./proc/elo_rating_spread_sheets/tube_test/tt_grouped_by_pairs_cage_cages-1-2-3-4-5-6_date_2022-09-06_2022-09-27.csv", index_col=0)
PAIRED_HCO = pd.read_csv("./proc/elo_rating_spread_sheets/home_cage_observation/hco_grouped_by_pairs_cage_cages-2-4-5_date_2022-09-07_2022-10-05.csv", index_col=0)


In [12]:
ELO_RATING_SPREADSHEET_OUTPUT_DIRECTORY = os.path.join(OUTPUT_DIR, "elo_rating_spread_sheets")

In [13]:
os.makedirs(ELO_RATING_SPREADSHEET_OUTPUT_DIRECTORY, exist_ok=True)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [14]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names



In [15]:
# SCORING_DF = pd.read_excel(RAW_DATA_FILE_PATH, sheet_name=INPUTTED_SHEET_NAME, header=ALL_HEADER_ROW)

In [16]:
# weights_df = pd.read_excel(RAW_DATA_FILE_PATH, header=[0])
weights_df = pd.read_csv(WEIGHTS_PATH, header=[0,1])
weights_df = weights_df.set_index([('cage', 'cage'), ('id', 'id')])
# Stacking the level 0 of the columns (Group1, Group2), 
# which will move it to the index, creating a new row for each unique value in level 0
weights_df = weights_df.stack(level=1)
weights_df = weights_df.reset_index()
weights_df = weights_df.rename(columns={'level_2': 'date'})
weights_df["date"] = pd.to_datetime(weights_df["date"], format='%m/%d/%Y')
weights_df = weights_df.rename(columns={('cage', 'cage'): "cage", ('id', 'id'): "id"})
weights_df = weights_df.sort_values(by=["date", "cage", "id"])
weights_df = weights_df.reset_index(drop=True)

In [17]:
weights_df.columns

Index(['cage', 'id', 'date', 'weight', 'percent_body_weight', 'amount_fed',
       'Notes'],
      dtype='object')

In [18]:
weight_col = ['id', 'weight', 'percent_body_weight', 'amount_fed']

In [19]:
subject_weights_df = weights_df.copy()
subject_weights_df.columns = ["subject_{}".format(col) if col in weight_col else col for col in subject_weights_df.columns ]

agent_weights_df = weights_df.copy()
agent_weights_df.columns = ["agent_{}".format(col) if col in weight_col else col for col in agent_weights_df.columns  ]

In [20]:
subject_weights_df.head()

,cage,subject_id,date,subject_weight,subject_percent_body_weight,subject_amount_fed,Notes
0,1,1.1,2022-09-18,26.0,1.0,5.0,NaN
1,1,1.2,2022-09-18,25.2,1.0,5.0,NaN
2,1,1.3,2022-09-18,24.8,1.0,5.0,NaN
3,1,1.4,2022-09-18,24.7,1.0,5.0,NaN
4,2,2.1,2022-09-18,29.5,1.0,5.0,NaN


In [21]:
agent_weights_df.head()

,cage,agent_id,date,agent_weight,agent_percent_body_weight,agent_amount_fed,Notes
0,1,1.1,2022-09-18,26.0,1.0,5.0,NaN
1,1,1.2,2022-09-18,25.2,1.0,5.0,NaN
2,1,1.3,2022-09-18,24.8,1.0,5.0,NaN
3,1,1.4,2022-09-18,24.7,1.0,5.0,NaN
4,2,2.1,2022-09-18,29.5,1.0,5.0,NaN


In [22]:
all_elo_score_history_df = []
for path in ELO_RATING_HISTORY_GLOB:
    all_elo_score_history_df.append(pd.read_csv(path, index_col=0))
all_elo_score_history_df = pd.concat(all_elo_score_history_df)
all_elo_score_history_df["date"] = pd.to_datetime(all_elo_score_history_df["date"], format='%Y-%m-%d')
all_elo_score_history_df = all_elo_score_history_df.sort_values(by=["date", "total_match_number"])
all_elo_score_history_df = all_elo_score_history_df.reset_index(drop=True)

In [23]:
all_elo_score_history_df.columns

Index(['total_trial_number', 'total_match_number', 'subject_id', 'agent_id',
       'original_elo_rating', 'updated_elo_rating', 'win_draw_loss',
       'subject_ranking', 'agent_ranking', 'pairing_index', 'index', 'date',
       'cage', 'box', 'match', 'tuple_animal_id', 'trial', 'winner',
       'keep_row', 'match_is_tie', 'trial_number', 'loser',
       'session_number_difference', 'strain', 'experiment_type', 'observer',
       'notes', 'length_of_observations', 'cage_#', 'action', 'sheet_name',
       'left_number_of_spots', 'right_number_of_spots',
       'spot_number_difference', 'percent_difference', 'runner'],
      dtype='object')

In [24]:
all_elo_score_history_df = pd.merge(left=all_elo_score_history_df, right=subject_weights_df, left_on=["subject_id", "date"], right_on=["subject_id", "date"], how="left")

all_elo_score_history_df = pd.merge(left=all_elo_score_history_df, right=agent_weights_df, left_on=["agent_id", "date"], right_on=["agent_id", "date"], how="left")


In [25]:
all_elo_score_history_df.columns

Index(['total_trial_number', 'total_match_number', 'subject_id', 'agent_id',
       'original_elo_rating', 'updated_elo_rating', 'win_draw_loss',
       'subject_ranking', 'agent_ranking', 'pairing_index', 'index', 'date',
       'cage_x', 'box', 'match', 'tuple_animal_id', 'trial', 'winner',
       'keep_row', 'match_is_tie', 'trial_number', 'loser',
       'session_number_difference', 'strain', 'experiment_type', 'observer',
       'notes', 'length_of_observations', 'cage_#', 'action', 'sheet_name',
       'left_number_of_spots', 'right_number_of_spots',
       'spot_number_difference', 'percent_difference', 'runner', 'cage_y',
       'subject_weight', 'subject_percent_body_weight', 'subject_amount_fed',
       'Notes_x', 'cage', 'agent_weight', 'agent_percent_body_weight',
       'agent_amount_fed', 'Notes_y'],
      dtype='object')

In [26]:
all_elo_score_history_df = pd.merge(left=all_elo_score_history_df, right=PAIRED_HCO, left_on="tuple_animal_id", right_on="hco_tuple_animal_id", how="left")
all_elo_score_history_df = pd.merge(left=all_elo_score_history_df, right=PAIRED_RC, left_on="tuple_animal_id", right_on="rc_tuple_animal_id", how="left")
all_elo_score_history_df = pd.merge(left=all_elo_score_history_df, right=PAIRED_TT, left_on="tuple_animal_id", right_on="tt_tuple_animal_id", how="left")

In [27]:
all_elo_score_history_df = all_elo_score_history_df.drop(columns=["total_match_number", "total_trial_number", "index", "cage_x", "box", "match", "keep_row", "session_number_difference", "observer", "notes", "length_of_observations", "sheet_name", "runner", "cage_y", "cage", "Notes_y", "Notes_x", 'processed_cage_number', 'match_is_tie', 'trial'], errors="ignore")

In [28]:
all_elo_score_history_df.columns

Index(['subject_id', 'agent_id', 'original_elo_rating', 'updated_elo_rating',
       'win_draw_loss', 'subject_ranking', 'agent_ranking', 'pairing_index',
       'date', 'tuple_animal_id', 'winner', 'trial_number', 'loser', 'strain',
       'experiment_type', 'cage_#', 'action', 'left_number_of_spots',
       'right_number_of_spots', 'spot_number_difference', 'percent_difference',
       'subject_weight', 'subject_percent_body_weight', 'subject_amount_fed',
       'agent_weight', 'agent_percent_body_weight', 'agent_amount_fed',
       'hco_tuple_animal_id', 'hco_winner', 'hco_loser', 'hco_averaged_winner',
       'hco_averaged_loser', 'hco_averaged_winner_win_count',
       'hco_averaged_loser_win_count', 'hco_count_difference',
       'hco_match_count', 'hco_percent_win', 'hco_percentage_tie',
       'rc_tuple_animal_id', 'rc_winner', 'rc_loser',
       'rc_average_number_of_switches', 'rc_winner_no_ties',
       'rc_loser_no_ties', 'rc_averaged_winner', 'rc_averaged_loser',
       'r

In [29]:
# Step 2: Specify the columns to move to the left
cols_to_move = ['strain', 'experiment_type', 'date', 'cage_#', 'tuple_animal_id', 'subject_id', 'agent_id', 
                'winner', 'loser', 'win_draw_loss', 'original_elo_rating', 'updated_elo_rating', 
                'subject_ranking', 'agent_ranking',  'subject_weight', 'subject_percent_body_weight',
                'subject_amount_fed', 'agent_weight', 'agent_percent_body_weight', 'agent_amount_fed']

# Step 3: Get the list of other columns
other_cols = [col for col in all_elo_score_history_df.columns if col not in cols_to_move]

# Step 4: Concatenate the lists to get the new column order
new_col_order = cols_to_move + other_cols


In [30]:
all_elo_score_history_df = all_elo_score_history_df[new_col_order]

In [31]:
all_elo_score_history_df.head()

,strain,experiment_type,date,cage_#,tuple_animal_id,subject_id,agent_id,winner,loser,win_draw_loss,...,tt_winner,tt_loser,tt_averaged_winner,tt_averaged_loser,tt_averaged_winner_win_count,tt_averaged_loser_win_count,tt_count_difference,tt_match_count,tt_percent_win,tt_percentage_tie
0,C57,tube_test,2022-09-06,1.0,"('1.1', '1.2')",1.1,1.2,1.1,1.2,1.0,...,"['1.1', '1.1', '1.1', '1.1', '1.1', '1.1', '1....","['1.2', '1.2', '1.2', '1.2', '1.2', '1.2', '1....",1.1,1.2,8,0,8,8,1.000,False
1,C57,tube_test,2022-09-06,1.0,"('1.1', '1.2')",1.2,1.1,1.1,1.2,0.0,...,"['1.1', '1.1', '1.1', '1.1', '1.1', '1.1', '1....","['1.2', '1.2', '1.2', '1.2', '1.2', '1.2', '1....",1.1,1.2,8,0,8,8,1.000,False
2,C57,tube_test,2022-09-06,2.0,"('2.1', '2.2')",2.1,2.2,2.1,2.2,1.0,...,"['2.1', '2.1', '2.1', '2.1', '2.1', '2.1', '2....","['2.2', '2.2', '2.2', '2.2', '2.2', '2.2', '2....",2.1,2.2,7,1,6,8,0.875,False
3,C57,tube_test,2022-09-06,2.0,"('2.1', '2.2')",2.2,2.1,2.1,2.2,0.0,...,"['2.1', '2.1', '2.1', '2.1', '2.1', '2.1', '2....","['2.2', '2.2', '2.2', '2.2', '2.2', '2.2', '2....",2.1,2.2,7,1,6,8,0.875,False
4,C57,tube_test,2022-09-06,3.0,"('3.1', '3.2')",3.2,3.1,3.2,3.1,1.0,...,"['3.2', '3.2', '3.2', '3.2', '3.2', '3.2', '3....","['3.1', '3.1', '3.1', '3.1', '3.1', '3.1', '3....",3.2,3.1,8,0,8,8,1.000,False


In [32]:
all_elo_score_history_df.columns

Index(['strain', 'experiment_type', 'date', 'cage_#', 'tuple_animal_id',
       'subject_id', 'agent_id', 'winner', 'loser', 'win_draw_loss',
       'original_elo_rating', 'updated_elo_rating', 'subject_ranking',
       'agent_ranking', 'subject_weight', 'subject_percent_body_weight',
       'subject_amount_fed', 'agent_weight', 'agent_percent_body_weight',
       'agent_amount_fed', 'pairing_index', 'trial_number', 'action',
       'left_number_of_spots', 'right_number_of_spots',
       'spot_number_difference', 'percent_difference', 'hco_tuple_animal_id',
       'hco_winner', 'hco_loser', 'hco_averaged_winner', 'hco_averaged_loser',
       'hco_averaged_winner_win_count', 'hco_averaged_loser_win_count',
       'hco_count_difference', 'hco_match_count', 'hco_percent_win',
       'hco_percentage_tie', 'rc_tuple_animal_id', 'rc_winner', 'rc_loser',
       'rc_average_number_of_switches', 'rc_winner_no_ties',
       'rc_loser_no_ties', 'rc_averaged_winner', 'rc_averaged_loser',
       'r

In [33]:
all_elo_score_history_df.to_csv("./all_elo_score.csv")

# Adding the distance for reward port code

In [34]:
raise ValueError()

ValueError: 

In [ ]:
# Getting the sheet names for the excel file
xls = pd.ExcelFile(RAW_DATA_FILE_PATH)
raw_data_sheet_names = xls.sheet_names

In [ ]:
raw_data_sheet_names

In [ ]:
# Going through each sheet and creating a dataframe of it
sheet_name_to_everything = defaultdict(dict)
for sheet in raw_data_sheet_names:
    sheet_name_to_everything[sheet]["SCORING_DF"] = pd.read_excel(RAW_DATA_FILE_PATH, sheet_name=sheet, header=ALL_HEADER_ROW)

## Standarizing the Columns

- Making all the column names lower case and removing any extra spaces in the beginning and at the end
    - One dictionary per sheet

In [ ]:
for key, value in sheet_name_to_everything.items():
    # Creating a dictionary that maps the original column name to the standarized one
    column_name_to_standarized = defaultdict(dict)
    for col in value["SCORING_DF"]:
        # Making the column name lower case and removing the spaces
        column_name_to_standarized[col] = "_".join(str(col).lower().strip().split(" "))
    value["column_name_to_standarized"] = column_name_to_standarized

In [ ]:
value["column_name_to_standarized"]

In [ ]:
# Renaming all the columns to the lower case and space removed version
for key, value in sheet_name_to_everything.items():
    value["processed_behavior_recording_dataframe"] = value["SCORING_DF"].rename(columns=value["column_name_to_standarized"])
    value["processed_behavior_recording_dataframe"]["sheet_name"] = key

In [ ]:
value["processed_behavior_recording_dataframe"].head()

## Dropping All Rows without a Winner

- Because we will be filling all empty cells with the previous value, we must remove all rows that don't have any data. This is usually cells that don't have any winners or losers filled in the row.
    

# **NOTE**: If there is a set column that is used for the winner and loser,  the rows up by session, then enter it in the cell below between the quotation marks. Default is `"winner"` and `"loser"`

In [ ]:
for key, value in sheet_name_to_everything.items():
    # Dropping all rows that don't have any values in the winner column
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].dropna(subset=WINNER_ID_COLUMN)
    
    # Dropping all rows that don't have any values in the loser column
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].dropna(subset=LOSER_ID_COLUMN)
    
    # Getting all the floats from the strings, removing any spaces and other characters
    value["processed_behavior_recording_dataframe"][WINNER_ID_COLUMN] = value["processed_behavior_recording_dataframe"][WINNER_ID_COLUMN].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)
    
    value["processed_behavior_recording_dataframe"][LOSER_ID_COLUMN] = value["processed_behavior_recording_dataframe"][LOSER_ID_COLUMN].astype(str).apply(lambda x: re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x)[0] if re.findall(r"[-+]?(?:\d*\.\d+|\d+)", x) else x)

## Keeping only the rows with scorable actions

- Specific for home cage observations. This protocol records the type of action that occured between an initiator and reciever. Only certain actions(usually fighting and chasing) are counted towards points for the ELO score. So we will remove all other rows that contain different actions

In [ ]:
for key, value in sheet_name_to_everything.items():   
    sheet_name_to_everything[key]["processed_behavior_recording_dataframe"][ALL_ACTION_COLUMN] = sheet_name_to_everything[key]["processed_behavior_recording_dataframe"][ALL_ACTION_COLUMN].apply(lambda x: x.strip())
    
    # Keeping all rows that have the inputted action
    sheet_name_to_everything[key]["processed_behavior_recording_dataframe"] = sheet_name_to_everything[key]["processed_behavior_recording_dataframe"][sheet_name_to_everything[key]["processed_behavior_recording_dataframe"][ALL_ACTION_COLUMN].isin(ALL_SCORABLE_ACTION)]

copy_of_sheet_name_to_everything = sheet_name_to_everything.copy()
# Removing keys if the dataframe is empty    
for key, value in copy_of_sheet_name_to_everything.items():
    if sheet_name_to_everything[key]["processed_behavior_recording_dataframe"].empty:
        sheet_name_to_everything.pop(key, None)

## Adding the session number

- We are adding the session number to all the trials. The session number is counting the number of recording sessions that have happened up until that trial. Usually, each session in the spreadsheet is divided up by a session's first row having the date filled in. So we will label a new session when a date is filled in.

# NOTE: If there is a set column that divides the rows up by session, then enter it in the cell below between the quotation marks. Default is `"date"`

In [ ]:
for key, value in sheet_name_to_everything.items():
    current_processed_behavior_recording_dataframe = value["processed_behavior_recording_dataframe"].copy()
    value["processed_behavior_recording_dataframe"][ALL_SESSION_DIVIDER_COLUMN] = value["processed_behavior_recording_dataframe"][ALL_SESSION_DIVIDER_COLUMN].fillna(method='ffill')



In [ ]:
value["processed_behavior_recording_dataframe"].head()

# Getting the Session number differences

- Getting the indexes of where each new session starts. So that we can add the session number to each row.

In [ ]:
for key, value in sheet_name_to_everything.items():

    # Filling all the empty cells with the value in the previous cell
    value["processed_behavior_recording_dataframe"] = value["processed_behavior_recording_dataframe"].fillna(method='ffill')

    value["processed_behavior_recording_dataframe"]["session_number_difference"] = value["processed_behavior_recording_dataframe"][ALL_SESSION_DIVIDER_COLUMN].astype('category').cat.codes.diff()


In [ ]:
value["processed_behavior_recording_dataframe"].head(n=15)

In [ ]:
value["processed_behavior_recording_dataframe"].tail(n=15)

## Getting the cage number

- The cage number is sometimes not consistent throughout a spreadsheet for the same cage. So we will try to standarize it into one value.
    - **NOTE**: This assumes cage numbers are actual numbers. And not entirely consisting of letters. If that isn't the case, then you must edit this cell for your needs.

# NOTE: If there is a column that has the cage number information, replace the `None` with the column name in quotation marks

- Change this when you have id to cage

In [ ]:
sheet_name_to_everything['CAGE2']["processed_behavior_recording_dataframe"]

## Calculating Elo rating

- Example calculation

In [ ]:
calculation.calculate_elo_rating(subject_elo_rating=1000, agent_elo_rating=2000, score=1)

In [ ]:
calculation.update_elo_rating(winner_id="A", loser_id="B")

## Get the Elo rating for all the events

- Going through each row or interaction and calculating the new Elo rating for the winner and loser. This will create a new dataframe based off of the processed behavioral recording dataframe

# NOTE: If there are a set of columns to keep, edit the cell below with the name of the columns each in quotation marks seperated by commas
   - i.e. `['runner', 'date', 'match', 'winner', 'loser', 'notes', 'session_number',
       'session_number_difference']`

In [ ]:
all_columns_to_keep = None

In [ ]:
for key, value in sheet_name_to_everything.items():
    # Calculating the Elo rating    
    value["index_to_elo_rating_and_meta_data"] = calculation.iterate_elo_rating_calculation_for_dataframe(dataframe=value["processed_behavior_recording_dataframe"], winner_id_column=WINNER_ID_COLUMN, loser_id_column=LOSER_ID_COLUMN, additional_columns=value["processed_behavior_recording_dataframe"].columns)

     # Making a dataframe from the dictionary 
    value["elo_rating_dataframe"] = pd.DataFrame.from_dict(value["index_to_elo_rating_and_meta_data"], orient="index")

    value["elo_rating_dataframe"]["cage_#"] = re.findall(r'\d+', key)[0]


In [ ]:
value["elo_rating_dataframe"].head()

In [ ]:
value["elo_rating_dataframe"].tail()

In [ ]:
value["elo_rating_dataframe"].groupby("id").count()

## Combining all the Elo rating dataframes into one

In [ ]:
# Putting all the dataframes into one list
all_sheet_elo_scord_dataframe_list = []
for key, value in sheet_name_to_everything.items():    
    all_sheet_elo_scord_dataframe_list.append(value["elo_rating_dataframe"])

# Combining all the dataframes into one
all_sheet_elo_scord_dataframe_combined = pd.concat(all_sheet_elo_scord_dataframe_list)

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

- Adding the strain information

In [ ]:
all_ids = set(all_sheet_elo_scord_dataframe_combined["id"].unique()).union(set(all_sheet_elo_scord_dataframe_combined["agent_id"].unique()))

# NOTE: If there are strains that are associated to each cage, then create a dictionary of cage numbers to strains inside the `{}`
- i.e. `CAGE_TO_STRAIN = {"1": "C57", "2": "C57", "3": "C57", "4": "CD1", "5": "CD1", "6": "CD1"}`

In [ ]:
all_sheet_elo_scord_dataframe_combined["strain"] = all_sheet_elo_scord_dataframe_combined["cage_#"].map(CAGE_TO_STRAIN)

- Adding the name of the experiment

In [ ]:
# Adding the name of the experiment
all_sheet_elo_scord_dataframe_combined["experiment_type"] = PROTOCOL_NAME

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

In [ ]:
# Checking to see how many rows for each subject in each cage
all_sheet_elo_scord_dataframe_combined.groupby(['id','cage_#']).count()

## Creating a dataframe with only the final Elo rating for each subject

In [ ]:
# Checking to see which cage and subject combination has more than one row
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_combined.groupby(['id','cage_#']).size().reset_index()
all_sheet_elo_scord_dataframe_groupby = all_sheet_elo_scord_dataframe_groupby[all_sheet_elo_scord_dataframe_groupby[0] >= 1]

# Going through each combination and saving the combination to a dictionary
index_to_id_and_processed_cage_number = defaultdict(dict)
for index, row in all_sheet_elo_scord_dataframe_groupby.iterrows():
    index_to_id_and_processed_cage_number[index]['id'] = row['id']
    index_to_id_and_processed_cage_number[index]['cage_#'] = row['cage_#']


In [ ]:
index_to_id_and_processed_cage_number

- Getting the final Elo rating for each cage and subject combination

In [ ]:
for index, (key, value) in enumerate(index_to_id_and_processed_cage_number.items()):   
    # The Elo rating dataframe for each cage and subject combination
    per_subject_dataframe = all_sheet_elo_scord_dataframe_combined[(all_sheet_elo_scord_dataframe_combined["id"] == value["id"]) & (all_sheet_elo_scord_dataframe_combined["cage_#"] == value["cage_#"])]
    # Getting the final Elo rating for each combination
    # -1 Means that we're getting the data from the last row

    index_to_id_and_processed_cage_number[index]["final_elo_rating"] = per_subject_dataframe.iloc[-1]["updated_elo_rating"]
    
    index_to_id_and_processed_cage_number[index]["strain"] = per_subject_dataframe.iloc[-1]["strain"]



In [ ]:
id_to_final_elo_rating_df = pd.DataFrame.from_dict(index_to_id_and_processed_cage_number, orient="index")
# Adding protocol name
id_to_final_elo_rating_df["experiment_type"] = PROTOCOL_NAME
# Adding rank
id_to_final_elo_rating_df["rank"] = id_to_final_elo_rating_df.groupby("cage_#")["final_elo_rating"].rank("dense", ascending=False)
# Sorting by cage and then id
id_to_final_elo_rating_df = id_to_final_elo_rating_df.sort_values(by=['cage_#', "id"], ascending=True).reset_index(drop=True)

In [ ]:
id_to_final_elo_rating_df.head()

In [ ]:
id_to_final_elo_rating_df.tail()

# Making plots for all sheets

- Getting the dates the files were being recorded to use for the file name

In [ ]:
all_sheet_elo_scord_dataframe_combined.head()

In [ ]:
all_sheet_elo_scord_dataframe_combined.tail()

- Getting the earliest and the latest dates for all the recordings

In [ ]:
all_earlist_dates = []
all_latest_dates = []
for key, value in sheet_name_to_everything.items():
    try:
        # Getting all the earliest dates for each sheet
        all_earlist_dates.append(value["elo_rating_dataframe"][ALL_SESSION_DIVIDER_COLUMN].min())
        all_latest_dates.append(value["elo_rating_dataframe"][ALL_SESSION_DIVIDER_COLUMN].max())
    except:
        print("WARNING: {} does not have dates as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
value["elo_rating_dataframe"]

In [ ]:
try:
    # Turning the Dates into a easier to read format
    # Getting the 0th part of split to remove seconds
    earliest_date = str(min(all_earlist_dates)).split()[0]
    latest_date = str(max(all_latest_dates)).split()[0]
    print("Earlist date: {}".format(earliest_date))
    print("Latest date: {}".format(latest_date))
except:
    earliest_date = None
    latest_date = None

- Getting the cage numbers

In [ ]:
all_cages_list = []
# Creating a list of all the cage numbers
for key, value in sheet_name_to_everything.items():
    try:
        for cage in value["elo_rating_dataframe"]["cage_#"].unique():
            all_cages_list.append(cage)
    except:
        print("WARNING: {} does not have cage number as columns".format(key))
        warnings.warn("Look at warning from above or below")

In [ ]:
try:
    all_cages_string = "-".join(sorted([sheet.lower().strip("cage").strip() for sheet in all_cages_list]))
    all_cages_string = "cages-{}".format(all_cages_string)
    print("String of cage names to use for file name: {}".format(all_cages_string))
except: 
    warnings.warn("WARNING: There are no cage numbers to make a title out of")
    all_cages_string = None

- Creating an output directory to save the plots

In [ ]:
plot_output_directory = os.path.join(".", "proc", "plots", "{}_elo_rating".format(PROTOCOL_NAME))

In [ ]:
plot_output_directory

In [ ]:
os.makedirs(plot_output_directory, exist_ok=True)

# **NOTE**: Sometimes this cell needs to be run again to make sure the size is correct

In [ ]:
# Getting the highest and lowest Elo rating for cutoffs of the Y-axis
max_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].max()
min_elo_rating = all_sheet_elo_scord_dataframe_combined["updated_elo_rating"].min()

plt.rcParams["figure.figsize"] = (13.5,7.5)
# Making a plot for each sheet
for key, value in sheet_name_to_everything.items():
    # Setting the size of the figure
    plt.rcParams["figure.figsize"] = (13.5,7.5)
    print(key)
    elo_rating_dataframe = value["elo_rating_dataframe"]
    # Using a new figure template for each sheet
    fig, ax = plt.subplots()        
        
    try:
        # Drawing vertical lines that represent when each session begins
        # Based on when a row has a different session than the previous row
        for index, row in elo_rating_dataframe[elo_rating_dataframe['session_number_difference'].astype(bool)].iterrows():
            # Offsetting by 0.5 to avoid drawing the line on the dot
            # Drawing the lines a little above the max and a little below the minimum
            plt.vlines(x=[row["total_match_number"] - 0.5], ymin=min_elo_rating-50, ymax=max_elo_rating+50, colors='black', linestyle='dashed')
    except:
        print("WARNING: {} does not have a column for session divider".format(key))
        warnings.warn("Look at warning from above or below")
            
    # Drawing a line for each subject
    for subject in sorted(elo_rating_dataframe["id"].unique()):
        # Getting all the rows with the current subject
        subject_dataframe = elo_rating_dataframe[elo_rating_dataframe["id"] == subject]
        # Making the current match number the X-Axis
        plt.plot(subject_dataframe["total_match_number"], subject_dataframe["updated_elo_rating"], '-o', label=subject)

    # Labeling the X/Y Axis and the title
    ax.set_xlabel("Trial Number")
    ax.set_ylabel("Elo rating")
    # Formattnig Cohort and Experiment Name so that it's more readable with spacing and capitalization
    
    ax.set_title("{} Elo Rating for {}".format(PROTOCOL_NAME, key))
    
    # To show the legend
    ax.legend(loc="upper left")
    # Setting the values of the Y-axis
    plt.ylim(min_elo_rating-50, max_elo_rating+50) 
    # Saving the plot
    file_name_parts_separated = [PREFIX_NAME, key, earliest_date, latest_date]
    file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])
    
    file_name_full = "elo_rating_{}.png".format(file_name_parts_combined)
    # Removing all the spaces and replacing them with underscores
    file_name_full = "_".join(file_name_full.split(" "))
    plt.savefig(os.path.join(plot_output_directory, file_name_full))
    # Showing the plots
    plt.show()

# Saving the Dataframes

- Creating a subfolder to put the Elo rating Spreadsheets

In [ ]:
elo_rating_spreadsheet_output_directory = os.path.join(".", "proc", "elo_rating_spread_sheets", "{}".format(PROTOCOL_NAME))

In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
os.makedirs(elo_rating_spreadsheet_output_directory, exist_ok=True)

- Saving the dataframes to a file

In [ ]:
file_name_parts_separated = [all_cages_string, PREFIX_NAME, earliest_date, latest_date]
file_name_parts_combined = "_".join([part for part in file_name_parts_separated if part])

file_name_full = "{}_elo-rating-history.csv".format(file_name_parts_combined)
print(file_name_full)
all_sheet_elo_scord_dataframe_combined.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

In [ ]:
file_name_full = "{}_final-elo-rating.csv".format(file_name_parts_combined)
print(file_name_full)
id_to_final_elo_rating_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name_full))

# Seeing which subject is the dominant or submissive

- Grouping all the rows with the same pair

In [ ]:
all_processed_behavior_recording_list = []
for key, value in sheet_name_to_everything.items():
    all_processed_behavior_recording_list.append(value["processed_behavior_recording_dataframe"])
    

- Combining all the dataframes from all the cages

In [ ]:
all_processed_behavior_recording_df = pd.concat(all_processed_behavior_recording_list)

In [ ]:
all_processed_behavior_recording_df.head()

- Getting a tuple of the animal IDs to be able to group

# Note: This assumes that all the animal IDs are different across cages and that all IDs are numbers. i.e. there are no duplicate IDs in different cages.

In [ ]:
# Getting the animal IDs from the Match string
# i.e. Getting all the floats and removing all spaces
# Sorting so that the animal IDs are always in the same order
all_processed_behavior_recording_df["animal_id"] =  all_processed_behavior_recording_df.apply(lambda x: sorted([x["winner"], x["loser"]]), axis=1)


In [ ]:
# Making a tuple out of the list
# Tuples are used because lists are mutable and can't be grouped with
all_processed_behavior_recording_df["tuple_animal_id"] = all_processed_behavior_recording_df["animal_id"].apply(lambda x: tuple(x))

In [ ]:
all_processed_behavior_recording_df.head()

- Removing columns that would be unnecessary for the pairings

In [ ]:
all_processed_behavior_recording_df.columns

In [ ]:
# Getting only the columns that we need
all_processed_behavior_recording_df = all_processed_behavior_recording_df[['date', 'winner', 'loser', 'animal_id', 'tuple_animal_id', "cage_#"]]

In [ ]:
all_processed_behavior_recording_df.head()

- Getting the ID of the winner and the loser for each pair with each match

In [ ]:
all_wins_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['winner'].apply(list)
all_loses_per_pair = all_processed_behavior_recording_df.groupby("tuple_animal_id")['loser'].apply(list)

In [ ]:
all_wins_per_pair[:5]

- Making a dataframe of all the winner IDs and all the loser IDs for a given pair

In [ ]:
all_competition_per_pair_df = pd.concat([all_wins_per_pair, all_loses_per_pair], axis=1).reset_index()

In [ ]:
all_competition_per_pair_df = all_competition_per_pair_df.rename(columns={k: PREFIX_NAME + "_" + k for k in all_competition_per_pair_df.columns})

In [ ]:
all_competition_per_pair_df

- Adding the cage information

In [ ]:
# Getting the cage number for each pair
dropped_duplicate_all_processed_behavior_recording_df = all_processed_behavior_recording_df[["tuple_animal_id"]].drop_duplicates()

In [ ]:
dropped_duplicate_all_processed_behavior_recording_df.head()

- Calculating the overall winner and loser. Also seeing if there is signficant difference in the number of wins to see if one is dominant over the other

In [ ]:
all_competition_per_pair_df["{}_averaged_winner".format(PREFIX_NAME)] = all_competition_per_pair_df["{}_winner".format(PREFIX_NAME)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_averaged_loser".format(PREFIX_NAME)] = all_competition_per_pair_df["{}_loser".format(PREFIX_NAME)].apply(lambda x: Counter(x).most_common(1)[0][0])
all_competition_per_pair_df["{}_winner_count".format(PREFIX_NAME)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(PREFIX_NAME)].count(x["{}_averaged_winner".format(PREFIX_NAME)]), axis=1)
all_competition_per_pair_df["{}_loser_count".format(PREFIX_NAME)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner".format(PREFIX_NAME)].count(x["{}_averaged_loser".format(PREFIX_NAME)]), axis=1)
all_competition_per_pair_df["{}_count_difference".format(PREFIX_NAME)] = all_competition_per_pair_df["{}_winner_count".format(PREFIX_NAME)] - all_competition_per_pair_df["{}_loser_count".format(PREFIX_NAME)]
all_competition_per_pair_df["{}_match_count".format(PREFIX_NAME)] = all_competition_per_pair_df["{}_winner".format(PREFIX_NAME)].apply(lambda x: len(x))
all_competition_per_pair_df["{}_percent_win".format(PREFIX_NAME)] = all_competition_per_pair_df.apply(lambda x: x["{}_winner_count".format(PREFIX_NAME)] / x["{}_match_count".format(PREFIX_NAME)], axis=1)
all_competition_per_pair_df["{}_percentage_tie".format(PREFIX_NAME)] = all_competition_per_pair_df["{}_percent_win".format(PREFIX_NAME)].apply(lambda x: True if x < 0.75 else False)

In [ ]:
all_competition_per_pair_df

- Saving the competiton pair results dataframe to a file

In [ ]:
file_name = "{}_grouped_by_pairs_cage_{}_date_{}_{}.csv".format(PREFIX_NAME, all_cages_string, earliest_date, latest_date)


In [ ]:
elo_rating_spreadsheet_output_directory

In [ ]:
file_name

In [ ]:
all_competition_per_pair_df.to_csv(os.path.join(elo_rating_spreadsheet_output_directory, file_name))

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a4490980-3f6a-4f44-80eb-ebd789a5b21f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>